# Large MNL work

Sam Maurer, April 2018 | Python 3.6

This notebook is for development, testing, and demonstration of the template for MNL with large numbers of alternatives.

In [ ]:
import os; os.chdir('../')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca

In [ ]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [ ]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

# Large MNL

In [ ]:
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings']
m.choice_column = 'unit_id'
m.alt_sample_size = 10
m.chooser_filters = ['household_id % 1000 < 1']

m.model_expression = 'res_price_per_sqft - 1'

m.name = 'large-mnl-test'
m.tags = ['sam', 'testing']

In [ ]:
%%time
m.fit()

In [ ]:
m.fitted_parameters

In [ ]:
m.register()

In [ ]:
mm.get_step('large-mnl-test').fitted_parameters

In [ ]:
m.out_chooser_filters = ['household_id % 1000 == 17']

In [ ]:
#%%time
#m.run()

In [ ]:
#m.choices